In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import datetime
import json

In [3]:
import tweepy
from config import consumer_key, consumer_secret, access_token, access_token_secret
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
api

In [4]:
data_path    = "../data/"
input_file   = "twitter_accounts.xlsx"
output_file1 = "twitter_users.xlsx"
output_file2 = "twitter_tweets.xlsx"
#output_file3 = "twitter_users_twaccount.csv"
#output_file4 = "twitter_tweets_twaccount.csv"

#output_file3 = "Twitter_Tweets_Text.csv"

In [5]:
### FUNCTION: Get Twitter User Info ###
def get_user_info(user_account):

    # Check user_name format
    if (user_account[0] != '@'):
        return NULL
        
    # Get user info from the Json object
    try:
        obj_user = api.get_user(user_account)
        # Get the specific column data
        user_id                = obj_user["id_str"]
        user_name              = obj_user["name"]
        user_screen_name       = obj_user["screen_name"]
        user_verified          = obj_user["verified"]
        user_created_at        = obj_user["created_at"]
        user_location          = obj_user["location"]
        user_followers_count   = obj_user["followers_count"]
        user_friends_count     = obj_user["friends_count"]
        user_listed_count      = obj_user["listed_count"]
        user_favourites_count  = obj_user["favourites_count"]
        user_statuses_count    = obj_user["statuses_count"]
        user_description       = obj_user["description"]

    except tweepy.TweepError as e:
        print(f"exception for {user_account}: {e}")

    # Date/Time Transformation
    converted_time   = datetime.datetime.strptime(user_created_at, "%a %b %d %H:%M:%S %z %Y")
    user_create_time = converted_time.strftime("%Y-%m-%d %H:%M:%S")
        
    # Return the user info needed
    user_info = {
        "user_id"                : user_id,
        "user_account"           : user_account,
        "user_name"              : user_name,
        "user_screen_name"       : user_screen_name,
        "user_verified"          : user_verified,
        "user_created_at"        : user_create_time,
        "user_location"          : user_location,
        "user_followers_count"   : user_followers_count,
        "user_friends_count"     : user_friends_count,
        "user_listed_count"      : user_listed_count,
        "user_favourites_count"  : user_favourites_count,
        "user_statuses_count"    : user_statuses_count,
        "user_description"       : user_description
    }
    return user_info

In [6]:
#user_1 = get_user_info("@CrackedEggFirst")
#print(user_1)

In [7]:
### FUNCTION: Get Twitter User Info ###
def get_user_tweets(user_account, count_tweets=200):
    
    # Check user_name format
    if ((user_account[0] != '@') or not(count_tweets >= 1 and count_tweets <= 1000)):
        return NULL
    
    # Get the tweets by the user
    user_tweets = api.user_timeline(user_account, count=count_tweets)
    #print(len(user_tweets))
    
    # Prepare for the dataframe
    tweets_id             = []
    tweets_user_id        = []
    tweets_user_account   = []
    tweets_created_at     = []
    tweets_longitude      = []
    tweets_latitude       = []
    tweets_quote_count    = []
    tweets_reply_count    = []
    tweets_retweet_count  = []
    tweets_favorite_count = []
    tweets_lang           = []
    tweets_text           = []
    
    Business_id           = []
    idx = twitter_accounts.index(user_account)
    Yelp_id = str(Yelp_business_ids[idx])

    # For each tweet
    for tweet in user_tweets:
        # Get the tweet's info from the Json object
        tweet_id                        = tweet["id_str"]
        tweet_user_id                   = tweet["user"]["id_str"]
        tweet_user_acount               = user_account
        tweet_created_at                = tweet["created_at"]
        #tweet_quote_count               = tweet["quote_count"]
        tweet_quote_count               = ""
        #tweet_reply_count               = tweet["reply_count"]
        tweet_reply_count               = ""
        tweet_retweet_count             = tweet["retweet_count"]
        tweet_favorite_count            = tweet["favorite_count"]
        tweet_lang                      = tweet["lang"]
        tweet_text                      = tweet["text"]

        #print(json.dumps(tweet["coordinates"], indent=4, sort_keys=True))
        if tweet["coordinates"]:
            tweet_longitude = tweet["coordinates"]["coordinates"][0]
            tweet_latitude  = tweet["coordinates"]["coordinates"][1]
        else:
            tweet_longitude = ""
            tweet_latitude  = ""
        
        # Date/Time Transformation
        converted_time = datetime.datetime.strptime(tweet_created_at, "%a %b %d %H:%M:%S %z %Y")
        tweet_time     = converted_time.strftime("%Y-%m-%d %H:%M:%S")

        # Add the info of the tweet into the dataframe
        tweets_id.append(tweet_id)
        tweets_user_id.append(tweet_user_id)
        tweets_user_account.append(tweet_user_acount)
        tweets_created_at.append(tweet_time)
        tweets_longitude.append(tweet_longitude)
        tweets_latitude.append(tweet_latitude)
        tweets_quote_count.append(tweet_quote_count)
        tweets_reply_count.append(tweet_reply_count)
        tweets_retweet_count.append(tweet_retweet_count)
        tweets_favorite_count.append(tweet_favorite_count)
        tweets_lang.append(tweet_lang)
        tweets_text.append(tweet_text)
        Business_id.append(Yelp_id)

    # Return: tweets info
    df_tweets_info = pd.DataFrame({
        "Business_id"    : Business_id,
        "twitter_account"   : tweets_user_account,
        "tweet_id"       : tweets_id,
        #"user_id"        : tweets_user_id,
        "created_at"     : tweets_created_at,
        #"quote_count"    : tweets_quote_count,
        #"reply_count"    : tweets_reply_count,
        "retweet_count"  : tweets_retweet_count,
        "favorite_count" : tweets_favorite_count,
        "lang"           : tweets_lang,
        #"longitude"      : tweets_longitude,
        #"latitude"       : tweets_latitude
        "text"           : tweets_text
    })
    return df_tweets_info

In [8]:
#user1_tweets = get_user_tweets("@CarsonKitchen", 1)

In [9]:
# Get Twitter accounts from the Input file
df_twitter_accounts = pd.read_excel(data_path+input_file)
df_twitter_accounts.head()

,business_id,name,state,stars,Twitter_account
0,_v7lMUtdd6WlgKUEBLA_VA,Don Ramon Mexican Restau,OH,4.0,@DonRamons
1,_VHU9KZUHRLp2Hc2JOKlYg,Cioppino Restaurant & Ci,PA,4.0,@CioppinoPGH
2,_Vi33ZAf0hdBn5Cu0AnQig,Zoes Kitchen,AZ,4.0,@ZoesKitchen
3,-2Arz8twKJmxHMS3Sv-TeA,Steep & Brew West,WI,3.5,@SteepNBrewWest
4,-49WY_TEa9ZEcRk_GnuLog,Cracker Barrel Old Count,OH,3.5,@CrackerBarrel


In [10]:
twitter_accounts  = df_twitter_accounts["Twitter_account"].tolist()
Yelp_business_ids = df_twitter_accounts["business_id"].tolist()
Yelp_user_names   = df_twitter_accounts["name"].tolist()
Yelp_states       = df_twitter_accounts["state"].tolist()

#len(df_twitter_accounts)

In [11]:
dic = {}
for uc in twitter_accounts:
    if uc in dic:
        dic[uc] += 1
    else:
        dic[uc] = 1

for di in dic:
    if dic[di] > 1:
        print(di, dic[di])

len(twitter_accounts)

80

In [12]:
yid = []
nas = []
for user_account in twitter_accounts:
    user_account = str(user_account)
    if user_account[0]=="@":
        idx = twitter_accounts.index(user_account)
        yid.append(str(Yelp_business_ids[idx]))
        nas.append(str(Yelp_user_names[idx]))
    #print(f'{Yelp_id} *** {user_account}')
    
print(len(yid), len(nas))
print(yid[0:3])
print(nas[0:3])

80 80
['_v7lMUtdd6WlgKUEBLA_VA', '_VHU9KZUHRLp2Hc2JOKlYg', '_Vi33ZAf0hdBn5Cu0AnQig']
['Don Ramon Mexican Restau', 'Cioppino Restaurant & Ci', 'Zoes Kitchen']


In [13]:
# Create Dataframe of Twitter User Info

Business_id = []
users_name  = []
users_id               = []
users_account          = []
users_screen_name      = []
users_city             = []
users_state            = []
users_country          = []
users_verified         = []
users_created_at       = []
users_location         = []
users_followers_count  = []
users_friends_count    = []
users_listed_count     = []
users_favourites_count = []
users_statuses_count   = []
users_description      = []

for user_account in twitter_accounts:
    user_account = str(user_account)

    if (user_account[0]=="@"):
        idx = twitter_accounts.index(user_account)
        Business_id.append(str(Yelp_business_ids[idx]))
        users_name.append(str(Yelp_user_names[idx]))
        users_state.append(str(Yelp_states[idx]))

        # Get Twitter User info
        user_info = get_user_info(user_account)
        users_id.append(user_info["user_id"])
        users_account.append(user_info["user_account"])
        #users_name.append(user_info["user_name"])
        users_screen_name.append(user_info["user_screen_name"])
        #users_location.append(user_info["user_location"])
        users_description.append(user_info["user_description"])
        users_verified.append(user_info["user_verified"])
        users_followers_count.append(user_info["user_followers_count"])
        users_friends_count.append(user_info["user_friends_count"])
        users_listed_count.append(user_info["user_listed_count"])
        users_favourites_count.append(user_info["user_favourites_count"])
        users_statuses_count.append(user_info["user_statuses_count"])
        users_created_at.append(user_info["user_created_at"])
        
        #location = user_info["user_location"].split(',')
        #if len(location)>=2:
        #    users_city.append(user_info["user_location"].replace(f',{location[-1]}', "").strip())
        #    users_state.append(location[-1].strip())
        #else:
        #    users_city.append(user_info["user_location"].strip())
        #    users_state.append("")
        users_city.append(user_info["user_location"])
        users_country.append("United States")

# New format required by James
df_twitter_users = pd.DataFrame ({
    #"user_id"                : users_id,
    "Business_id"            : Business_id,
    "twitter_account"           : users_account,
    "name"                   : users_name,
    #"user_name"              : users_name,
    "screen_name"            : users_screen_name,
    #"user_location"          : users_location,
    "city"                   : users_city,
    "state"                  : users_state,
    "country"                : users_country,
    "verified"               : users_verified,
    "followers_count"        : users_followers_count,
    "friends_count"          : users_friends_count,
    "listed_count"           : users_listed_count,
    "favourites_count"       : users_favourites_count,
    "statuses_count"         : users_statuses_count,
    #"user_description"       : users_description,
    "created_at"             : users_created_at
})
df_twitter_users.head()

,Business_id,twitter_account,name,screen_name,city,state,country,verified,followers_count,friends_count,listed_count,favourites_count,statuses_count,created_at
0,_v7lMUtdd6WlgKUEBLA_VA,@DonRamons,Don Ramon Mexican Restau,DonRamons,415-864-2700,OH,United States,False,244,487,17,2271,1436,2011-05-15 00:47:12
1,_VHU9KZUHRLp2Hc2JOKlYg,@CioppinoPGH,Cioppino Restaurant & Ci,CioppinoPGH,"Pittsburgh, PA",PA,United States,False,914,502,11,751,1016,2017-01-17 20:40:27
2,_Vi33ZAf0hdBn5Cu0AnQig,@ZoesKitchen,Zoes Kitchen,ZoesKitchen,,AZ,United States,True,22946,31,439,9697,16780,2009-05-28 15:38:34
3,-2Arz8twKJmxHMS3Sv-TeA,@SteepNBrewWest,Steep & Brew West,SteepNBrewWest,"6656 Odana Road., Madison, WI",WI,United States,False,112,387,7,94,770,2013-07-09 17:13:55
4,-49WY_TEa9ZEcRk_GnuLog,@CrackerBarrel,Cracker Barrel Old Count,CrackerBarrel,,OH,United States,True,119240,305,529,7392,10081,2011-05-18 22:15:26


In [14]:
len(df_twitter_users)

80

In [15]:
df_twitter_users.head()

,Business_id,twitter_account,name,screen_name,city,state,country,verified,followers_count,friends_count,listed_count,favourites_count,statuses_count,created_at
0,_v7lMUtdd6WlgKUEBLA_VA,@DonRamons,Don Ramon Mexican Restau,DonRamons,415-864-2700,OH,United States,False,244,487,17,2271,1436,2011-05-15 00:47:12
1,_VHU9KZUHRLp2Hc2JOKlYg,@CioppinoPGH,Cioppino Restaurant & Ci,CioppinoPGH,"Pittsburgh, PA",PA,United States,False,914,502,11,751,1016,2017-01-17 20:40:27
2,_Vi33ZAf0hdBn5Cu0AnQig,@ZoesKitchen,Zoes Kitchen,ZoesKitchen,,AZ,United States,True,22946,31,439,9697,16780,2009-05-28 15:38:34
3,-2Arz8twKJmxHMS3Sv-TeA,@SteepNBrewWest,Steep & Brew West,SteepNBrewWest,"6656 Odana Road., Madison, WI",WI,United States,False,112,387,7,94,770,2013-07-09 17:13:55
4,-49WY_TEa9ZEcRk_GnuLog,@CrackerBarrel,Cracker Barrel Old Count,CrackerBarrel,,OH,United States,True,119240,305,529,7392,10081,2011-05-18 22:15:26


In [16]:
# Create Dataframe of Tweets

count_user = 0
for user_account in twitter_accounts:
    user_account = str(user_account)

    if (user_account[0]=="@"):
        df_tweets_info = get_user_tweets(user_account, 200)
        
        if (count_user == 0):
            df_total_tweets_info = df_tweets_info.copy()
        else:
            df_total_tweets_info = df_total_tweets_info.append(df_tweets_info)
            
        count_user += 1

print(count_user)

80


In [17]:
print(len(df_total_tweets_info))
df_total_tweets_info.head()

15011


,Business_id,twitter_account,tweet_id,created_at,retweet_count,favorite_count,lang,text
0,_v7lMUtdd6WlgKUEBLA_VA,@DonRamons,1029239981104017408,2018-08-14 05:35:08,179,0,en,RT @SFGiants: Let those feelings out\n\n #Beat...
1,_v7lMUtdd6WlgKUEBLA_VA,@DonRamons,1008148314599010304,2018-06-17 00:44:23,0,1,en,@amymarquez Yeah! Enjoy 🌮🥑🍹
2,_v7lMUtdd6WlgKUEBLA_VA,@DonRamons,903591448792195073,2017-09-01 12:12:43,1,2,en,#Margarita ☀️🔥holiday weekend loooonng lunch k...
3,_v7lMUtdd6WlgKUEBLA_VA,@DonRamons,895171391590477825,2017-08-09 06:34:25,2,1,en,#Fogust ☀️🥑🌮🍹#SpiceItUp Enjoy #SanFrancisco be...
4,_v7lMUtdd6WlgKUEBLA_VA,@DonRamons,890903925737566208,2017-07-28 11:57:02,3,2,en,Enjoy #summer 🥑🌮☀️#SanFrancisco #tacos #comfo...


In [18]:
df_total_tweets_info.tail()

,Business_id,twitter_account,tweet_id,created_at,retweet_count,favorite_count,lang,text
194,XQETjKH84gxDrAiz2lH7Wg,@milkbarstore,1125791604604854272,2019-05-07 15:56:49,3,32,en,Same. https://t.co/IlmXJGggeg
195,XQETjKH84gxDrAiz2lH7Wg,@milkbarstore,1125515701601566721,2019-05-06 21:40:28,3,54,en,"Defeated the Night King, ordered cake to celeb..."
196,XQETjKH84gxDrAiz2lH7Wg,@milkbarstore,1125079019920211968,2019-05-05 16:45:15,1,0,en,RT @emspillar: had @milkbarstore birthday cake...
197,XQETjKH84gxDrAiz2lH7Wg,@milkbarstore,1124805907039363078,2019-05-04 22:40:00,2,2,en,@YukiSNYC Hi! So sorry about that please email...
198,XQETjKH84gxDrAiz2lH7Wg,@milkbarstore,1124730665881231360,2019-05-04 17:41:01,1,48,en,The care package for every occasion. Including...


In [19]:
tweets_id_chk = df_total_tweets_info["tweet_id"].tolist()

dic = {}
for uc in tweets_id_chk:
    if uc in dic:
        dic[uc] += 1
    else:
        dic[uc] = 1

for di in dic:
    if dic[di] > 1:
        print(di, dic[di])

len(tweets_id_chk)

15011

In [20]:
# Output Files
df_twitter_users.to_excel(data_path + output_file1, index=False, header=True)
df_total_tweets_info.to_excel(data_path + output_file2, index=False, header=True)

In [21]:
# # Output Files
# df_twitter_users_4others = df_twitter_users.drop("user_account", axis=1)
# df_twitter_users_4others.to_csv(data_path + output_file1, index=False, header=True)

# df_total_tweets_info_4others = df_total_tweets_info.drop("user_account", axis=1)
# df_total_tweets_info_4others.to_csv(data_path + output_file2, index=False, header=True)